In [ ]:
import scanpy as sc
import numpy as np
import squidpy as sq


In [5]:
adata_msi = sc.read_h5ad("/lustre/groups/ml01/workspace/eirini.giannakoulia/datasets/MSI_concat_neurotransmitters.h5ad")

/ictstr01/home/icb/eirini.giannakoulia/tools/apps/mamba/envs/vitatrack/lib/python3.9/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [6]:
adata_msi

AnnData object with n_obs × n_vars = 5443 × 1538
    obs: 'technology', 'clusters', 'random_split', 'half_split', 'slide', 'msi_concat_clusters'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection', 'spatialy_highly_variable', 'msi_highly_variable'
    uns: 'hvg', 'msi_concat_clusters', 'neighbors', 'pca', 'slide_colors', 'umap'
    obsm: 'X_pca', 'X_umap', 'spatial', 'spatial_warp'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [ ]:
# Step 1: Convert m/z to numeric and bin similar features
adata_msi = adata_msi[:, adata_msi.var_names.astype(float) > 288.15].copy()
# adata_msi.var_names = adata_msi.var_names.astype(float)
# bin_size = 0.05  
# adata_msi.var["binned_mz"] = np.round(adata_msi.var_names / bin_size) * bin_size
# adata_msi = adata_msi[:, adata_msi.var.groupby("binned_mz").apply(lambda x: x.index[x.X.mean(axis=0).idxmax()])]


/ictstr01/home/icb/eirini.giannakoulia/tools/apps/mamba/envs/vitatrack/lib/python3.9/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [ ]:
# Step 2: Normalize per-pixel intensities
sc.pp.normalize_total(adata_msi, target_sum=1e4)

# Step 3: Filter out low-variance and background noise features
sc.pp.filter_genes(adata_msi, min_disp=0.1)
adata_msi = adata_msi[:, adata_msi.X.mean(axis=0) > (0.01 * adata_msi.X.max())]

# Step 4: Spatial feature selection using Moran’s I
sq.gr.spatial_autocorr(adata_msi, mode="moran")
adata_msi = adata_msi[:, adata_msi.var["moranI_score"] > 0.2]

# Step 5: Remove batch effects (if applicable)
sc.pp.combat(adata_msi)

# Step 6: Log transform for better downstream analysis
sc.pp.log1p(adata_msi)
